In [1]:
# Using test data to build/prototype-out a ML model
    # Further restructuring data for ML input layers
        # Variable isolation and selection refinement
    # Reshaping, encoding, normalization
    # Input, layer, and output specification
    # Force keras/tensorflow to use GPU backend

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM, GRU
from keras.models import load_model
from keras import backend as K

#num_cores = 4
#config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        #inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        #device_count = {'CPU' : 1, 'GPU' : 1})
#session = tf.Session(config=config)
#K.set_session(session)

# Preprocessing Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,OneHotEncoder
from keras.utils import to_categorical 

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

In [ ]:
# Mongodb connection for live feed of data into model
    # Implementation after verification of test data -> model prototype input layer working

In [ ]:
# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

In [4]:
# Read in test data for L2 orderbook state (bids + asks)
    # Read in test data for subsquent L2 orderbook update states (L2 updates to bid + asks)
snapshot_asks_df = pd.read_csv("test_data/snapshot_asks.csv")
snapshot_bids_df = pd.read_csv("test_data/snapshot_bids.csv")
l2update_df = pd.read_csv("test_data/l2update.csv", dtype ={'changes':object})
request_log_df= pd.read_csv("test_data/request_log.csv")

In [109]:
print(request_log_df.head())
print("===============================================")
print(snapshot_asks_df.head())
#snapshot_asks_df = snapshot_asks_df["side","price","size"]
print(snapshot_bids_df.head())
print("===============================================")
print(l2update_df.head())
print(l2update_df.tail())

      Unnamed: 0                       iso         epoch
0    request end  2018-04-17T01:05:03.469Z  1.523927e+09
1  request start  2018-04-17T00:55:03.354Z  1.523927e+09
   side    price       size
0  sell  8042.91  10.656147
1  sell  8042.92   0.001181
2  sell  8042.93   0.001305
3  sell  8042.94   0.001181
4  sell  8042.95   1.265966
  side    price      size
0  buy  8042.90  9.118009
1  buy  8042.83  0.002487
2  buy  8042.50  0.420000
3  buy  8042.00  0.001000
4  buy  8041.33  0.020000
  side    price   size                      time
0  buy  8041.33  0.000  2018-04-17T00:55:04.358Z
1  buy  8041.43  0.020  2018-04-17T00:55:04.375Z
2  buy  7940.12  0.000  2018-04-17T00:55:04.395Z
3  buy  8039.00  0.001  2018-04-17T00:55:04.412Z
4  buy  7972.56  0.000  2018-04-17T00:55:04.413Z
       side    price      size                      time
18088  sell  8078.07  0.000000  2018-04-17T01:05:03.247Z
18089   buy  8039.05  0.000000  2018-04-17T01:05:03.330Z
18090  sell  8040.52  0.005328  2018-04-

In [110]:
# Bid= buy
# Ask = sell

#L2 snapshot structure
    # [side,price,size]
    # 'side' added as part of structure for classification
    
#l2 updates structure
    # [side, price, size, time]

# Note on GDAX API about l2update structure:
    # size of "0" indicates the price level can be removed

In [5]:
#One-hot/categorical encoding test

#Join together buy and sell side of orderbook for encoding:
snapshot_both_df = pd.concat([snapshot_asks_df,snapshot_bids_df], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
snapshot_both_df.count()

side     28242
price    28242
size     28242
dtype: int64

In [6]:
# snapshot data cat/one-hot encoded:
data_s = snapshot_both_df.values
X_s = data_s[:,1:3]
y_s = data_s[:,0:1]
y_s = np.ravel(y_s)
label_encoder = LabelEncoder()
encoded_y_s = label_encoder.fit_transform(y_s)
onehot_encoder = OneHotEncoder(sparse=False)
encoded_y_s = encoded_y_s.reshape(len(encoded_y_s), 1)
onehot_y_s = onehot_encoder.fit_transform(encoded_y_s)

In [7]:
encoded_y_s,onehot_y_s,X_s

(array([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]], dtype=int64), array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[8042.91, 10.65614728],
        [8042.92, 0.00118097],
        [8042.93, 0.00130538],
        ...,
        [0.03, 7498.221],
        [0.02, 25955.001],
        [0.01, 153092.98092796]], dtype=object))

In [114]:
# L2 data cat/one-hot encoded:
data_l2 = l2update_df.values
X_l2 = data_l2[:,1:4]
y_l2 = data_l2[:,0:1]
y_l2 = np.ravel(y_l2)
label_encoder = LabelEncoder()
encoded_y_l2 = label_encoder.fit_transform(y_l2)
onehot_encoder = OneHotEncoder(sparse=False)
encoded_y_l2 = encoded_y_l2.reshape(len(encoded_y_l2), 1)
onehot_y_l2 = onehot_encoder.fit_transform(encoded_y_l2)

In [115]:
encoded_y_l2,onehot_y_l2,X_l2

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]], dtype=int64), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]), array([[8041.33, 0.0, '2018-04-17T00:55:04.358Z'],
        [8041.43, 0.02, '2018-04-17T00:55:04.375Z'],
        [7940.12, 0.0, '2018-04-17T00:55:04.395Z'],
        ...,
        [8040.52, 0.00532786, '2018-04-17T01:05:03.431Z'],
        [8039.06, 0.0, '2018-04-17T01:05:03.452Z'],
        [8039.0, 0.00375, '2018-04-17T01:05:03.486Z']], dtype=object))

In [10]:
# Test model build on snapshot state only 
    # L2 updates -> datetime string needs to be parsed/feed into LSTM design to properly work
model = Sequential()
model.add(Dense(units=4, activation='relu', input_dim=2))
model.add(Dense(units=5, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='tanh'))
model.add(Dense(units=2, activation='softmax'))

model.compile(optimizer='adam', 
              loss='logcosh', 
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 25        
_________________________________________________________________
dropout_1 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8         
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.fit(
    X_s,
    onehot_y_s,
    epochs=250,
    shuffle=True,
    verbose=2,
    batch_size=5000
)

Epoch 1/250
 - 0s - loss: 0.1273 - acc: 0.4636
Epoch 2/250
 - 0s - loss: 0.1262 - acc: 0.4958
Epoch 3/250
 - 0s - loss: 0.1252 - acc: 0.4933
Epoch 4/250
 - 0s - loss: 0.1240 - acc: 0.4965
Epoch 5/250
 - 0s - loss: 0.1231 - acc: 0.4922
Epoch 6/250
 - 0s - loss: 0.1221 - acc: 0.4936
Epoch 7/250
 - 0s - loss: 0.1211 - acc: 0.4997
Epoch 8/250
 - 0s - loss: 0.1204 - acc: 0.4978
Epoch 9/250
 - 0s - loss: 0.1197 - acc: 0.4956
Epoch 10/250
 - 0s - loss: 0.1190 - acc: 0.4981
Epoch 11/250
 - 0s - loss: 0.1185 - acc: 0.6018
Epoch 12/250
 - 0s - loss: 0.1180 - acc: 0.6018
Epoch 13/250
 - 0s - loss: 0.1177 - acc: 0.6018
Epoch 14/250
 - 0s - loss: 0.1173 - acc: 0.6018
Epoch 15/250
 - 0s - loss: 0.1169 - acc: 0.6018
Epoch 16/250
 - 0s - loss: 0.1166 - acc: 0.6018
Epoch 17/250
 - 0s - loss: 0.1163 - acc: 0.6018
Epoch 18/250
 - 0s - loss: 0.1161 - acc: 0.6018
Epoch 19/250
 - 0s - loss: 0.1158 - acc: 0.6018
Epoch 20/250
 - 0s - loss: 0.1156 - acc: 0.6018
Epoch 21/250
 - 0s - loss: 0.1155 - acc: 0.6018
E

Epoch 171/250
 - 0s - loss: 0.0809 - acc: 0.7901
Epoch 172/250
 - 0s - loss: 0.0807 - acc: 0.7917
Epoch 173/250
 - 0s - loss: 0.0803 - acc: 0.7949
Epoch 174/250
 - 0s - loss: 0.0809 - acc: 0.7910
Epoch 175/250
 - 0s - loss: 0.0807 - acc: 0.7917
Epoch 176/250
 - 0s - loss: 0.0802 - acc: 0.7922
Epoch 177/250
 - 0s - loss: 0.0802 - acc: 0.7954
Epoch 178/250
 - 0s - loss: 0.0796 - acc: 0.7969
Epoch 179/250
 - 0s - loss: 0.0794 - acc: 0.7965
Epoch 180/250
 - 0s - loss: 0.0802 - acc: 0.7930
Epoch 181/250
 - 0s - loss: 0.0803 - acc: 0.7909
Epoch 182/250
 - 0s - loss: 0.0800 - acc: 0.7934
Epoch 183/250
 - 0s - loss: 0.0804 - acc: 0.7915
Epoch 184/250
 - 0s - loss: 0.0795 - acc: 0.7962
Epoch 185/250
 - 0s - loss: 0.0804 - acc: 0.7929
Epoch 186/250
 - 0s - loss: 0.0789 - acc: 0.7982
Epoch 187/250
 - 0s - loss: 0.0801 - acc: 0.7896
Epoch 188/250
 - 0s - loss: 0.0787 - acc: 0.7967
Epoch 189/250
 - 0s - loss: 0.0795 - acc: 0.7929
Epoch 190/250
 - 0s - loss: 0.0796 - acc: 0.7934
Epoch 191/250
 - 0s 

In [53]:
#model.evaluate(X_s, onehot_y_s)
model_loss, model_accuracy=model.evaluate(X_s, onehot_y_s)
print(f"Test model - Loss: {model_loss}, Accuracy: {model_accuracy}")

28242/28242 [==============================] - 2s 64us/step
Test model - Loss: 0.032893436792366025, Accuracy: 0.9428510728701933


In [8]:
# L2 data cat/one-hot encoded with date removed:
data_l2 = l2update_df.values
X_l2 = data_l2[:,1:3]
y_l2 = data_l2[:,0:1]
y_l2 = np.ravel(y_l2)
label_encoder = LabelEncoder()
encoded_y_l2 = label_encoder.fit_transform(y_l2)
onehot_encoder = OneHotEncoder(sparse=False)
encoded_y_l2 = encoded_y_l2.reshape(len(encoded_y_l2), 1)
onehot_y_l2 = onehot_encoder.fit_transform(encoded_y_l2)
encoded_y_l2,onehot_y_l2,X_l2

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]], dtype=int64), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]), array([[8041.33, 0.0],
        [8041.43, 0.02],
        [7940.12, 0.0],
        ...,
        [8040.52, 0.00532786],
        [8039.06, 0.0],
        [8039.0, 0.00375]], dtype=object))

In [40]:
#Testing model on l2 data without timestamps:
np.set_printoptions(threshold=np.inf) # Reset: np.set_printoptions(threshold=1000)
model.predict_classes(X_l2)

ValueError: Error when checking : expected lstm_9_input to have 3 dimensions, but got array with shape (18093, 2)

In [55]:
### GRU/LSTM Modelload Test

#GRU data reshaping test with return_sequences = False
    # when True: input and output shape changes, currently not working
data = X_s.reshape((X_s.shape[0], 1, X_s.shape[1]))
#X_test =np.reshape(X_s,(X_s.shape[0],X_s.shape[1],1))
model = Sequential()
model.add(GRU(25,  input_shape = (1,2),return_sequences = False))
model.add(Dense(2))
model.add(Activation('sigmoid'))
    
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_13 (GRU)                 (None, 25)                2100      
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 52        
_________________________________________________________________
activation_18 (Activation)   (None, 2)                 0         
Total params: 2,152
Trainable params: 2,152
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(
    data,
    onehot_y_s,
    epochs=250,
    shuffle=True,
    verbose=2,
    batch_size=5000
)

Epoch 1/250
 - 1s - loss: 0.8543 - acc: 0.6018
Epoch 2/250
 - 0s - loss: 0.8488 - acc: 0.6018
Epoch 3/250
 - 0s - loss: 0.8432 - acc: 0.6018
Epoch 4/250
 - 0s - loss: 0.8376 - acc: 0.6018
Epoch 5/250
 - 0s - loss: 0.8321 - acc: 0.6018
Epoch 6/250
 - 0s - loss: 0.8266 - acc: 0.6018
Epoch 7/250
 - 0s - loss: 0.8214 - acc: 0.6018
Epoch 8/250
 - 0s - loss: 0.8161 - acc: 0.6018
Epoch 9/250
 - 0s - loss: 0.8110 - acc: 0.6018
Epoch 10/250
 - 0s - loss: 0.8060 - acc: 0.6018
Epoch 11/250
 - 0s - loss: 0.8010 - acc: 0.6018
Epoch 12/250
 - 0s - loss: 0.7959 - acc: 0.6018
Epoch 13/250
 - 0s - loss: 0.7884 - acc: 0.6018
Epoch 14/250
 - 0s - loss: 0.7825 - acc: 0.6018
Epoch 15/250
 - 0s - loss: 0.7773 - acc: 0.6018
Epoch 16/250
 - 0s - loss: 0.7718 - acc: 0.6031
Epoch 17/250
 - 0s - loss: 0.7647 - acc: 0.6088
Epoch 18/250
 - 0s - loss: 0.7593 - acc: 0.6097
Epoch 19/250
 - 0s - loss: 0.7281 - acc: 0.6099
Epoch 20/250
 - 0s - loss: 0.6934 - acc: 0.6104
Epoch 21/250
 - 0s - loss: 0.6910 - acc: 0.6109
E

Epoch 171/250
 - 0s - loss: 0.6745 - acc: 0.6018
Epoch 172/250
 - 0s - loss: 0.6744 - acc: 0.6018
Epoch 173/250
 - 0s - loss: 0.6742 - acc: 0.6018
Epoch 174/250
 - 0s - loss: 0.6741 - acc: 0.6018
Epoch 175/250
 - 0s - loss: 0.6739 - acc: 0.6018
Epoch 176/250
 - 0s - loss: 0.6738 - acc: 0.6018
Epoch 177/250
 - 0s - loss: 0.6737 - acc: 0.6018
Epoch 178/250
 - 0s - loss: 0.6736 - acc: 0.6018
Epoch 179/250
 - 0s - loss: 0.6735 - acc: 0.6018
Epoch 180/250
 - 0s - loss: 0.6734 - acc: 0.6018
Epoch 181/250
 - 0s - loss: 0.6733 - acc: 0.6018
Epoch 182/250
 - 0s - loss: 0.6732 - acc: 0.6018
Epoch 183/250
 - 0s - loss: 0.6731 - acc: 0.6018
Epoch 184/250
 - 0s - loss: 0.6731 - acc: 0.6018
Epoch 185/250
 - 0s - loss: 0.6730 - acc: 0.6018
Epoch 186/250
 - 0s - loss: 0.6730 - acc: 0.6018
Epoch 187/250
 - 0s - loss: 0.6729 - acc: 0.6018
Epoch 188/250
 - 0s - loss: 0.6728 - acc: 0.6018
Epoch 189/250
 - 0s - loss: 0.6728 - acc: 0.6018
Epoch 190/250
 - 0s - loss: 0.6728 - acc: 0.6018
Epoch 191/250
 - 0s 

# 

In [28]:
model_loss, model_accuracy=model.evaluate(data, onehot_y_s)
print(f"Test model - Loss: {model_loss}, Accuracy: {model_accuracy}")

28242/28242 [==============================] - 2s 81us/step
Test model - Loss: 0.6724440935152582, Accuracy: 0.6018341477232491
